In [45]:
# !pip install einops

In [46]:
import pandas as pd
import os
from tqdm.notebook import tqdm
path = 'combined_data'
header = ['timestamp', 'voc2', 'no2', 'eth', 'co', 'temp', 'pressure', 'humidity', 'mq3', 'mq7', 'mq9', 'mq135']
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [47]:
path

'combined_data'

In [48]:
def checkHeader(file_path):
    with open(file_path, 'r', newline='') as csvfile:
        # Read the first row of the CSV file
        first_row = csvfile.readline().strip()

        # Check if the first row matches the header
        if first_row != ','.join(header):
            # Read the rest of the file
            csvfile.seek(0)
            rows = csvfile.readlines()

            # Prepend the header to the rows
            rows.insert(0, ','.join(header) + '\n')

            # Write the updated rows back to the file
            with open(file_path, 'w', newline='') as csvfile:
                csvfile.writelines(rows)

In [49]:
fold_path = path
save_path = 'combined_data'
os.makedirs(save_path, exist_ok=True)
# for folder in os.listdir(fold_path):
#     print(folder)
#     dfs = []
#     for files in tqdm(os.listdir(os.path.join(fold_path, folder)), leave=False, desc= folder):
#         # print(os.path.join(fold_path, folder, files))
#         filepath = os.path.join(fold_path, folder, files)
#         checkHeader(filepath)
#         df = pd.read_csv(filepath, usecols=['timestamp', 'voc2', 'no2', 'eth', 'co', 'temp', 'pressure', 'humidity', 'mq3', 'mq7', 'mq9', 'mq135'])
#         dfs.append(df)

#     data = pd.concat(dfs, ignore_index=True)
#     data.sort_values(by=['timestamp'], inplace = True)
#     df = data['timestamp'].is_monotonic_increasing
#     print(f"Monotonically increasing {df} for {folder}")


#     data.to_csv(os.path.join(save_path, f"{folder}.csv"))




In [50]:
data_path = 'combined_data'

# st = set()
ls = os.listdir(data_path)
ls = [f.replace('.csv', '') for f in ls]
ls = [''.join(char for char in f if not char.isdigit()) for f in ls ]
ls = {key : value for value, key in enumerate(sorted(list(set(ls))))}

print(ls)
len(ls)

{'Background': 0, 'Cardamom': 1, 'Clinic_plus': 2, 'Coriander': 3, 'Cumin_Seeds': 4, 'Garlic': 5, 'Ginger': 6, 'Incense_stick': 7, 'Onion': 8, 'Polo': 9, 'Red_chilli': 10, 'Red_clilli': 11, 'Sanitizer': 12, 'Soap': 13, 'Turmeric': 14, 'Vicks': 15, 'Volini': 16}


17

In [51]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from einops import rearrange, repeat

In [52]:
class SmellDataset(Dataset):
  def __init__(self, data_path, t_lim=3000):

    # self.classToLabel = label
    self.data_path = data_path
    self.files = [f for f in os.listdir(data_path) if f.endswith('.csv')]
    # pprint(self.files)
    # checked works
    self.len = len(self.files)
    self.t_lim = t_lim
    ls = os.listdir(data_path)
    ls = [f.replace('.csv', '') for f in ls]
    ls = [''.join(char for char in f if not char.isdigit()) for f in ls ]
    ls = {key : value for value, key in enumerate(sorted(list(set(ls))))}


    self.classToLabel = ls

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    file_path = os.path.join(self.data_path, self.files[idx])
    df = pd.read_csv(file_path, nrows = self.t_lim)
    # df.drop_columns(['timestamp', ''])
    df.drop(columns=['timestamp', 'Unnamed: 0'],inplace = True)
    # pprint(df.head())

    fname = self.files[idx].split('.')[0]
    # key = [char for char in fname if char is not char.isdigit() else ]
    key = [char for char in fname if not char.isdigit() ]
    # print(key)
    key = "".join(key)
    # print(self.files[idx].split('.')[0])

    y = self.classToLabel[key]
    return df.to_numpy(dtype =np.float32), y



dataset = SmellDataset(data_path)
print(len(dataset))
loader = DataLoader(dataset, batch_size=8, shuffle=True, drop_last=True)


X, y = next(iter(loader))
print(X.shape, y.dtype)

# ((N, T, C) , N)

76
torch.Size([8, 3000, 11]) torch.int64


In [53]:
# X.dtype
# import matplotlib.pyplot as plt

# df = pd.read_csv(os.path.join(data_path, 'Vicks2.csv'))
# print(df.columns)
# ax = df.plot.hist(bins =100, column=['voc2', 'no2', 'eth', 'co', 'temp',
#        'pressure', 'humidity', 'mq3', 'mq7', 'mq9', 'mq135'], figsize=(10, 8))

- [x] DATA LOADER COMPLETE:
      # ((N, T, C) , N)
      

In [54]:
class CNN(nn.Module):
    def __init__(self, num_variables, num_classes):
        super().__init__()
        self.conv1 = nn.Conv1d(num_variables, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * (3000 // 2 // 2 // 2), 100)  # Adjust input_length as necessary
        self.fc2 = nn.Linear(100, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * (3000 // 2 // 2 // 2))  # Adjust input_length as necessary
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [55]:
# class Model1(nn.Module):
#   def __init__(self, input_size, hidden_size, num_layers, num_classes):
#     super().__init__()
#     self.hidden_size = hidden_size
#     self.num_layers = num_layers
#     self.num_classes = num_classes
#     self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
#     self.fc = nn.Linear(hidden_size * num_layers, self.num_classes)

#   def forward(self, x):
#     output , h = self.gru(x)
#     h = rearrange(h, 'd n c -> n (d c)')
#     return self.fc(h)

# m = Model1(11, 32, 1, 17).cuda()
# X = X.cuda()
# m(X).shape

In [57]:
num_variables = 11
num_classes = 17  # Labels range from 0 to 16
model = CNN(num_variables, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [60]:

X = X.to(device)
# print(X)
model(X).shape

RuntimeError: Given groups=1, weight of size [16, 11, 3], expected input[8, 3000, 11] to have 11 channels, but got 3000 channels instead

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.permute(0, 2, 1)  # Change shape to [batch_size, num_variables, sequence_length]
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')


In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.permute(0, 2, 1)  # Change shape to [batch_size, num_variables, sequence_length]
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')


In [ ]:
# Assuming train_loader and test_loader are your DataLoaders
num_epochs = 20
train_model(model, dataset, criterion, optimizer, num_epochs)
# evaluate_model(model, test_loader)


AttributeError: 'numpy.ndarray' object has no attribute 'permute'

In [ ]:
-np.log10(1/17)

In [ ]:
for X, y in loader:
  X = X.cuda()
  y = y.cuda()
  pred = model(X)
  pred = pred.softmax(dim=1)
  pred = torch.max(pred, dim=1).index

  acc = torch.float(pred == y).mean()

  print(acc)





In [ ]:
# prompt: iterate over loader and compute accuracy for each batch
acc_list = []

for X, y in loader:
  model.eval()
  X = X.cuda()
  y = y.cuda()
  with torch.no_grad():
    pred = model(X)
  pred = pred.softmax(dim=1)
  pred = torch.max(pred, dim=1).indices

  # print((pred == y).to(torch.float).mean())
  # break
  acc = (pred == y).to(torch.float).mean()
  acc_list.append(acc.item())
  print(acc)


accuracy = np.mean(acc_list)
print(f"Accuracy {accuracy}")
